In [914]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt  # Biblioteca para gerar gráficos
import pandas as pd
from sklearn import metrics, model_selection
from scipy import stats
import math


In [915]:
data = np.genfromtxt('breastcancer.csv', delimiter=',')

## Questão 1

Considere o conjunto de dados disponível em `breastcancer.csv`, organizado
em 31 colunas, sendo as 30 primeiras colunas os atributos e a última coluna a
saída. Os 30 atributos coletados de exames médicos são usados no diagnóstico
do câncer de mama, sendo 1 a classe positiva e 0 a classe negativa. Maiores 
detalhes sobre os dados podem ser conferidos em 
https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset.

a) Considerando uma validação cruzada em 10 folds, avalie modelos de classicação binária nos dados em questão. Para tanto, use as abordagens abaixo:
- Regressão logística (treinado com GD ou SGD);
- Análise do discriminante Gaussiano;
- Naive Bayes Gaussiano;

In [916]:
def k_fold_split(array, k = int):
    shuffled_data = np.random.permutation(array)
    folds = np.array_split(shuffled_data, k)
    return folds

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def GD(X, y, learning_rate, epochs):
    n = y.shape[0]
    # w = np.random.randn(X.shape[1])
    w = np.zeros(X.shape[1])

    for t in range(epochs):
        sum_e = np.zeros(X.shape[1])
        for i in range(n):
            ei = y[i] - sigmoid(np.dot(w.T, X[i]))
            sum_e += ei * X[i]
        w = w + learning_rate * (sum_e/n)
        # w = w + learning_rate * np.mean(np.dot(e, X))
    return w

folds = k_fold_split(data, 10)

results = []
for i, fold in enumerate(folds):
    test_set = fold
    training_sets = [x for j, x in enumerate(folds) if j != i]
    training_sets = np.vstack(training_sets)

    n_train = training_sets.shape[0]
    x_train = training_sets[:, np.arange(30)]
    x_train = stats.zscore(x_train)
    X_train = np.c_[np.ones(n_train), x_train]
    y_train = training_sets[:, 30]
    w_train = GD(X_train, y_train, 0.01, 100)

    n_test = test_set.shape[0]
    x_test = test_set[:, np.arange(30)]
    x_test = stats.zscore(x_test)
    X_test = np.c_[np.ones(n_test), x_test]
    y_test = test_set[:, 30]

    y_pred = sigmoid(np.dot(w_train, X_test.T))
    y_pred = np.around(y_pred)

    results.append(metrics.accuracy_score(y_test, y_pred))

results

print("acurácia média:", np.mean(results), ", desvio padrão:", np.std(results))

acurácia média: 0.9402255639097744 , desvio padrão: 0.03063416425779794


b) Para cada modelo criado, reporte valor médio e desvio padrão das métricas de **acurácia**, **revocação**, **precisão** e **F1-score**

In [917]:
# correct = 0
# for i, pred in enumerate(y_pred):
#     if pred == y[i]:
#         correct += 1
# print(correct)
# accuracy = correct / n
# accuracy
# metrics.accuracy_score(y, y_pred)